In [ ]:
#RDKit based DFT function to evaluate Free Energy in Chemical Structures generated from original and reorganized SMILES string #
#The code is initially generated with Openai's LLM i.e. "ChatGPT.com" #
#Then code is modified according to the need of the study #
#Initial code obtained on 12Dec2024 #


import os
from rdkit import Chem
from rdkit.Chem import AllChem

def calculate_free_energy(smiles):
    """
    Approximate free energy calculation using force field optimization.
    
    Parameters:
        smiles (str): SMILES notation of the molecule.

    Returns:
        float: Approximate free energy (arbitrary units), or None if calculation fails.
    """
    try:
        # Convert SMILES to RDKit molecule with explicit valence
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is None:
            raise ValueError("Invalid SMILES notation")

        # Sanitize the molecule manually to handle explicit valence
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL ^ Chem.SanitizeFlags.SANITIZE_PROPERTIES)

        # Add hydrogens and optimize geometry
        mol = Chem.AddHs(mol)
        if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) != 0:
            raise ValueError("Failed to embed molecule")
        if AllChem.UFFOptimizeMolecule(mol) != 0:
            raise ValueError("Failed to optimize molecule")

        # Calculate energy using UFF
        ff = AllChem.UFFGetMoleculeForceField(mol)
        energy = ff.CalcEnergy()

        return energy

    except Exception as e:
        print(f"Error calculating free energy for SMILES {smiles}: {e}")
        return None

def process_smiles_file(input_file, output_file):
    """
    Process a file containing SMILES notations, calculate free energy for each,
    and save results to an output file.

    Parameters:
        input_file (str): Path to input file containing SMILES notations.
        output_file (str): Path to output file for saving results.
    """
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            outfile.write("SMILES\tFree Energy (Approximate)\n")
            for line in infile:
                smiles = line.strip()
                if not smiles:
                    continue
                free_energy = calculate_free_energy(smiles)
                if free_energy is not None:
                    outfile.write(f"{smiles}\t{free_energy:.6f}\n")
                else:
                    outfile.write(f"{smiles}\tCalculation Failed\n")

    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    input_file = "InvalidAB.txt"  # Replace with your input file path
    output_file = "EnergysInvalidAB.txt"  # Replace with your desired output file path

    # Run the processing function
    process_smiles_file(input_file, output_file)
